# Timeseries Analysis with Batch Compute
__________________

This notebook will cover the typical pattern of using the Batch Compute API to scalably generate arbitrary timeseries data with Catalog. 

Here we will define a [`Function`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) that calculates daily Sentinel-2 NDVI statistics, masked to the Cropland Data Layer Hops class and summarized by week, in 4 major hop producing areas in the Northwestern United States.

The general methodology is as follows:
1. Read in a geojson of Census Tract boundary data and store as a [`Blob`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob)
2. Define a Python function that takes a GEOID and date as input parameters and:
    * Searches Sentinel-2 scenes over the specified date ranges
    * Mosaics Sentinel-2 Images and the Cropland Data Layer as ndarrays
    * Calculates NDVI and masks to the Hops class
    * Returns the mean NDVI value for the specified date. If no data is present, returns NaN
3. Submit the local function to [`Compute`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html) and monitor the status
4. Retrieve the time series results as a pandas DataFrame

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import Blob, Image, Product, properties as p
from descarteslabs.compute import Function, Job
from descarteslabs.exceptions import ConflictError

In [ ]:
import itertools
import json

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt

Defining global variables, including both Sentinel-2 ID, start and end dates, and a unique Function name:

In [ ]:
org = dl.auth.Auth().payload["org"]
user_id = dl.auth.Auth().namespace

In [ ]:
pid = "esa:sentinel-2:l2a:v1"
start_date = "2022-10-01"
end_date = "2022-11-01"
func_name = "Get NDVI Timeseries Values"
func_name

Next read in our input geojson file and save it as a blob in the next cell for reference in our function:

In [ ]:
gdf = gpd.read_file("data/hop_tracts.geojson")
gdf.plot()

#### **_Note on Saving Blobs:_** 

We do not always need to delete and overwrite our objects on every iteration as in the following cell. This notebook is designed for demonstration purposes where we do not care about preserving each prior version. 

In practice, as long as your blob has a **unique** ID you ignore the following cell and simply run:

    blob = Blob(name="unique-blob-name")
    blob.upload_data(json.dumps(gdf.to_json()))
    blob.save()

In [ ]:
try:
    # Create a new Blob object
    blob = Blob(
        name="hop_tracts",
        namespace="ndvi_timeseries_example_notebook",
        readers=[f"org:{org}"],
        tags=["examples"],
    )
    # Upload our DataFrame to this Blob:
    blob.upload_data(json.dumps(gdf.to_json()))
    blob.save()

except ConflictError:
    # Already exists within your org
    blob = Blob.get(name="hop_tracts", namespace="ndvi_timeseries_example_notebook")
    print("Blob already exists")
blob

## Function Methodology
These next few cells parse out the methodology contained within our function which will be defined below. 

The general steps are as follows:
1. Create an [`AOI`](https://docs.descarteslabs.com/descarteslabs/geo/readme.html#descarteslabs.geo.AOI) from our input geometry
2. Search for [Sentinel-2 L2A](https://app.descarteslabs.com/explorer/datasets/esa:sentinel-2:l2a:v1) [`ImageCollection`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.ImageCollection), filtered to the provided AOI and date range
3. Create an Image object of the USDA [Cropland Data Layer](https://app.descarteslabs.com/explorer/datasets/usda:cdl:v1) 2022 classification
4. Retrieve the associated __nir__ and __red__ bands for Sentinel-2 and class band for CDL as ndarrays
5. Calculate NDVI, mask to the hops class in the CDL array

Defining an AOI object out of one of our geojson features, alongside output raster metadata parameters (resolution, output CRS). Here we will look at the census tract for Moxee, WA:

In [ ]:
moxee_geom = gdf.loc[gdf["GEOID"] == "53077001702"]["geometry"].values[0]
aoi = dl.geo.AOI(moxee_geom, resolution=30.0, crs="EPSG:3857")
aoi

Searching Catalog for Sentinel-2 Imagery, chaining intersects and filter methods:

In [ ]:
s2_ic = (
    Product.get(pid)
    .images()
    .intersects(aoi)
    .filter("2022-10-01" <= p.acquired <= "2022-10-02")
    .sort("acquired")
    .limit(None)
).collect()
s2_ic

Retrieve a single Image from our 2022 Cropland Data Layer:

In [ ]:
cdl_image = Image.get("usda:cdl:v1:meta_2022_30m_cdls_v1")
cdl_image

#### _**Note on Metadata Limits**_: 

In the above example we are accessing a single CONUS-wide CDL [`Image`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html#descarteslabs.catalog.Image), as opposed to searching over a collection. In some cases this practice is optimal to avoid unnecessary search/filter steps as to avoid rate limits. [Please refer to the Quotas and Limits page for more details on metadata search and retrieval limits.](https://docs.descarteslabs.com/guides/quota.html)

Next we return our pixel data as ndarrays by mosaicking our imagery and retrieving the ndarray of our CDL image. Note, we are using the _**same AOI**_ in both cases, to ensure a matching geotransform:

In [ ]:
s2_arr = s2_ic.mosaic(bands=["nir", "red"])
cdl_arr = cdl_image.ndarray(bands=["class"], geocontext=s2_ic.geocontext)

s2_arr.shape, cdl_arr.shape

Calculate NDVI, mask to the Hop CDL value 56:

In [ ]:
# Calculating NDVI
nir = s2_arr[0, :, :]
red = s2_arr[1, :, :]
cdl = cdl_arr[0, :, :]

ndvi = (nir - red) / (nir + red)

hop_ndvi_msk = np.ma.masked_where(cdl != 56, ndvi)

Plotting our results:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 10))
ax[0].imshow(ndvi)
ax[0].set_title("NDVI")
ax[1].imshow(cdl, cmap="terrain")
ax[1].set_title("CDL")
ax[2].imshow(hop_ndvi_msk)
ax[2].set_title("NDVI - Hop Masked")
plt.tight_layout()

## Preparing the Compute Function
Now that we're settled on a methodology, we can put together our Python function to pass to Batch Compute. 

Here we will define our Python function. The steps have already been outlined in the above cells, just combined into a single function:

In [ ]:
def return_ndvi(geoid, date):
    import numpy as np
    import geopandas as gpd

    from datetime import datetime, timedelta
    from json import dumps, loads

    import descarteslabs as dl
    from descarteslabs.catalog import Blob, Image, Product, properties as p

    pid = "esa:sentinel-2:l2a:v1"
    cdl_pid = "usda:cdl:v1"

    # Getting this and next date
    date = datetime.strptime(date, "%Y-%m-%d")
    next_date = date + timedelta(days=1)

    # Retrieve our GDF from a Blob
    geom_blob = Blob.get(
        name="hop_tracts", namespace="ndvi_timeseries_example_notebook"
    )
    # Load our GDF
    geom_data = loads(geom_blob.data())
    gdf = gpd.read_file(geom_data)

    print("Pulled down GDF from Storage Blob")

    # Retrieve single geometry from GEOID passed
    in_geom = gdf[gdf["GEOID"] == geoid].iloc[0]["geometry"]

    # Create AOI from geometry
    aoi = dl.geo.AOI(in_geom, resolution=30.0, crs="EPSG:3857")

    # Search Sentinel2 for our date ranges

    print(f"Searching {date.strftime('%Y-%m-%d')} to {next_date.strftime('%Y-%m-%d')}")

    s2_ic = (
        Product.get(pid)
        .images()
        .intersects(aoi)
        .filter(date <= p.acquired < next_date)
        .sort("acquired")
        .limit(None)
    ).collect()

    # End if we have no imagery satisfying our filter conditions:
    try:
        assert len(s2_ic) > 0
    except:
        result_dict = {
            "GEOID": geoid,
            "mean_ndvi": np.nan,
            "date": date.strftime("%Y-%m-%d"),
        }
        print("No images found")
        return result_dict

    print(f"Found {len(s2_ic)} images")

    # Get CDL Image
    cdl_image = Image.get("usda:cdl:v1:meta_2022_30m_cdls_v1")

    ##Mosaic and ndarray our image data to the *same GeoContext*
    s2_arr = s2_ic.mosaic(bands=["nir", "red"], data_type="Float32", geocontext=aoi)
    cdl_arr = cdl_image.ndarray(bands=["class"], geocontext=aoi)

    print("Rastered imagery")

    # Calculating NDVI, then mask to Hop Class 56
    nir = s2_arr[0, :, :]
    red = s2_arr[1, :, :]
    cdl = cdl_arr[0, :, :]

    ndvi = (nir - red) / (nir + red)

    hop_ndvi_msk = np.ma.masked_where(cdl != 56, ndvi)

    # Returning mean
    mean_ndvi = float(hop_ndvi_msk.mean())

    print("Completed calculation")

    result_dict = {
        "GEOID": geoid,
        "mean_ndvi": mean_ndvi,
        "date": date.strftime("%Y-%m-%d"),
    }

    return result_dict

Next we'll define our input arguments for our function:
1. Generate a list of dates between our start and end dates
2. Generate a tuple of (GEOID, date) for each GEOID in our Census Tracts GDF

In [ ]:
date_list = pd.date_range(start_date, end_date).strftime("%Y-%m-%d").tolist()
geoids = gdf["GEOID"].unique().tolist()

In [ ]:
params = [args for args in itertools.product(geoids, date_list)]
len(params)

In [ ]:
params[0]

Let's test a run of this function locally, before we submit our function:

In [ ]:
return_ndvi(*params[10])

## Creating the Compute Function
We can now create our [`Function`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) object. 

Below we will pass several scaling parameters and save our function:

In [ ]:
async_func = Function(
    return_ndvi,
    name=func_name,
    image="python3.9:latest",
    cpus=0.5,
    memory=2,
    timeout=900,
    maximum_concurrency=50,
    retry_count=2,
)

async_func.save()
print(f"Saved {async_func.id}")

Now we can submit our arguments to our function and return a list of [`Job`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job)s:

In [ ]:
jobs = async_func.map(params)
len(jobs)

## Waiting for Completion
We now wait for our jobs to complete. There are several ways to do this, such as waiting programmatically via:

    async_func.wait_for_completion()

You can also track your progress at [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute)

## Retrieving Results
Once our function is completed, we can retrieve our result dictionaries as blobs by structuring the resulting IDs, as outlined in [01 Hello World.ipynb](01%20Hello%20World.ipynb):

    compute/{namespace}/{function-id}/{job-id}

Where the namespace is:

    {org}:{user-id}


In [ ]:
print(f"Results for {async_func.id}")
res_list = []
for b in (
    Blob.search()
    .filter(p.namespace == f"{org}:{user_id}")
    .filter(p.name.startswith(async_func.id))
    .filter(p.storage_type == "compute")
):
    print(f"ID: {b.id}")
    res_list.append(json.loads(b.data()))

Once all the results are returned, we can combine them into our timeseries dataset:

In [ ]:
res_df = pd.DataFrame(res_list)
fig, ax = plt.subplots()
res_df["date"] = pd.to_datetime(res_df["date"])

week_df = (
    res_df.groupby("GEOID")
    .resample("W-Mon", on="date")["mean_ndvi"]
    .mean()
    .reset_index()
)
for geoid, geoid_df in week_df.groupby("GEOID"):
    geoid_df.plot("date", "mean_ndvi", ax=ax, label=f"GEOID:{geoid}")
ax.set_title("NDVI");

## Cleanup
It is always best practice to clean up after ourselves!

In [ ]:
async_func.delete_jobs(delete_results=True)
async_func.delete()